# Excel - анализ 

Вопросы по смыслу полей.
- REGION_NM (регион РФ)
Регион по регистрации, пребыванию, почте?
- REGION_NM (регион РФ)
Регион ЦЕНТРАЛЬНЫЙ ОФИС - что это?
- LOAN_MAX_DLQ (номер максимальной просрочки, допущенной клиентом)
Что это?


Странные доходы PERSONAL_INCOME 
- 1950 руб. 23 года, пенсия. Возможно, инвалидность. Оставить как есть.
- 24 руб. Видимо, опечатка. Импутировать по среднему, или по среднему для 28-летних, военнослужащих по контракту.
- 6299,19 руб. Единственный доход с копейками. Возможно пенсия. Оставить как есть.

Наличие в заявке рабочего телефона GEN_PHONE_FL при отсутствии работы SOCSTATUS_WORK_FL.
- Доход похож на пенсию в 2011 году. Возможно, указано последнее место работы либо обман. Удалить или оставить до выяснения.

Наличие работы (GEN_INDUSTRY, GEN_TITLE) при отсутствии работы SOCSTATUS_WORK_FL. 
- Практически совпадает с "Наличие в заявке рабочего телефона при отсутствии работы". Доход похож на пенсию в 2011 году. Возможно, указано последнее место работы либо обман. Удалить или оставить до выяснения.

Странное (очень большое) время работы в месяцах WORK_TIME. 
- Исходя из максимального возраста в 67 лет, вряд ли стаж будет более 720 месяцев (60 лет х 12 месяцев) на последнем месте. Может быть, конечно, несколько совместительств. Обнулить всё, что больше 720, затем заменить средним.

Наличие пропусков в месте взятия кредита TP_PROVINCE при наличии кредита CREDIT.
- Клиент сказал сколько взял (см. CREDIT), но не сказал где? Можно предположить, что по месту фактического пребывания FACT_ADDRESS_PROVINCE. Соответственно, заменить пропуски 

REGION_NM - один пропуск, опечатка. 
- Заменить на ЗАПАДНО-СИБИРСКИЙ, т.к. Новосибирская область. Заменил в исходном csv файле

In [44]:
import os
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import roc_auc_score, accuracy_score

from sklearn.ensemble import GradientBoostingRegressor

In [45]:
os.chdir('D:\\ML\\Gewissta\\My course\\Credit OTP\\')

In [46]:
pd.set_option('display.max_columns', 100)
pd.set_option('display.max_info_columns',150)

In [47]:
df = pd.read_csv('Credit_OTP.csv', sep = ';', encoding='Windows-1251 ', decimal = ',')

In [48]:
df.head()

,AGREEMENT_RK,TARGET,AGE,SOCSTATUS_WORK_FL,SOCSTATUS_PENS_FL,GENDER,CHILD_TOTAL,DEPENDANTS,EDUCATION,MARITAL_STATUS,GEN_INDUSTRY,GEN_TITLE,ORG_TP_STATE,ORG_TP_FCAPITAL,JOB_DIR,FAMILY_INCOME,PERSONAL_INCOME,REG_ADDRESS_PROVINCE,FACT_ADDRESS_PROVINCE,POSTAL_ADDRESS_PROVINCE,TP_PROVINCE,REGION_NM,REG_FACT_FL,FACT_POST_FL,REG_POST_FL,REG_FACT_POST_FL,REG_FACT_POST_TP_FL,FL_PRESENCE_FL,OWN_AUTO,AUTO_RUS_FL,HS_PRESENCE_FL,COT_PRESENCE_FL,GAR_PRESENCE_FL,LAND_PRESENCE_FL,CREDIT,TERM,FST_PAYMENT,DL_DOCUMENT_FL,GPF_DOCUMENT_FL,FACT_LIVING_TERM,WORK_TIME,FACT_PHONE_FL,REG_PHONE_FL,GEN_PHONE_FL,LOAN_NUM_TOTAL,LOAN_NUM_CLOSED,LOAN_NUM_PAYM,LOAN_DLQ_NUM,LOAN_MAX_DLQ,LOAN_AVG_DLQ_AMT,LOAN_MAX_DLQ_AMT,PREVIOUS_CARD_NUM_UTILIZED
0,59910150,0,49,1,0,1,2,1,Среднее специальное,Состою в браке,Торговля,Рабочий,Частная компания,Без участия,Вспомогательный техперсонал,от 10000 до 20000 руб.,5000.0,Оренбургская область,Оренбургская область,Оренбургская область,Оренбургская область,ПОВОЛЖСКИЙ,1,1,1,1,1,0,0,0,0,0,0,0,8000.00,6,8650.0,0,1,220,18.0,0,0,1,1,1,6,2,1,1580.000000,1580.0,NaN
1,59910230,0,32,1,0,1,3,3,Среднее,Состою в браке,Торговля,Рабочий,Индивидуальный предприниматель,Без участия,Участие в основ. деятельности,от 10000 до 20000 руб.,12000.0,Кабардино-Балкария,Кабардино-Балкария,Кабардино-Балкария,Ставропольский край,ЮЖНЫЙ,1,1,1,1,0,0,0,0,0,0,0,0,21650.00,6,4000.0,0,1,137,97.0,1,0,1,1,1,6,1,1,4020.000000,4020.0,NaN
2,59910525,0,52,1,0,1,4,0,Неполное среднее,Состою в браке,Информационные технологии,Специалист,Государственная комп./учреж.,Без участия,Участие в основ. деятельности,от 10000 до 20000 руб.,9000.0,Иркутская область,Иркутская область,Иркутская область,Иркутская область,ВОСТОЧНО-СИБИРСКИЙ,1,1,1,1,1,0,0,0,1,0,0,0,33126.00,12,4000.0,0,1,251,84.0,0,0,1,2,1,11,0,0,0.000000,0.0,NaN
3,59910803,0,39,1,0,1,1,1,Высшее,Состою в браке,Образование,Руководитель среднего звена,Государственная комп./учреж.,Без участия,Участие в основ. деятельности,от 20000 до 50000 руб.,25000.0,Ростовская область,Ростовская область,Ростовская область,Ростовская область,ЮЖНЫЙ,0,1,0,0,1,1,0,0,0,0,0,0,8491.82,6,5000.0,0,0,36,168.0,1,1,1,1,1,6,3,1,1589.923333,1590.0,NaN
4,59911781,0,30,1,0,0,0,0,Среднее,Состою в браке,Государственная служба,Специалист,Государственная комп./учреж.,Без участия,Участие в основ. деятельности,от 10000 до 20000 руб.,12000.0,Кабардино-Балкария,Кабардино-Балкария,Кабардино-Балкария,Ставропольский край,ЮЖНЫЙ,1,1,1,1,0,0,0,0,1,0,0,0,21990.00,12,4000.0,0,1,83,101.0,1,0,1,2,1,16,2,1,1152.150000,2230.0,NaN


In [49]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 15223 entries, 0 to 15222
Data columns (total 52 columns):
 #   Column                      Non-Null Count  Dtype  
---  ------                      --------------  -----  
 0   AGREEMENT_RK                15223 non-null  int64  
 1   TARGET                      15223 non-null  int64  
 2   AGE                         15223 non-null  int64  
 3   SOCSTATUS_WORK_FL           15223 non-null  int64  
 4   SOCSTATUS_PENS_FL           15223 non-null  int64  
 5   GENDER                      15223 non-null  int64  
 6   CHILD_TOTAL                 15223 non-null  int64  
 7   DEPENDANTS                  15223 non-null  int64  
 8   EDUCATION                   15223 non-null  object 
 9   MARITAL_STATUS              15223 non-null  object 
 10  GEN_INDUSTRY                13856 non-null  object 
 11  GEN_TITLE                   13856 non-null  object 
 12  ORG_TP_STATE                13856 non-null  object 
 13  ORG_TP_FCAPITAL             138

In [50]:
# странно, почему PREVIOUS_CARD_NUM_UTILIZED float, а не int

In [51]:
df.isnull().sum()

AGREEMENT_RK                      0
TARGET                            0
AGE                               0
SOCSTATUS_WORK_FL                 0
SOCSTATUS_PENS_FL                 0
GENDER                            0
CHILD_TOTAL                       0
DEPENDANTS                        0
EDUCATION                         0
MARITAL_STATUS                    0
GEN_INDUSTRY                   1367
GEN_TITLE                      1367
ORG_TP_STATE                   1367
ORG_TP_FCAPITAL                1365
JOB_DIR                        1367
FAMILY_INCOME                     0
PERSONAL_INCOME                   0
REG_ADDRESS_PROVINCE              0
FACT_ADDRESS_PROVINCE             0
POSTAL_ADDRESS_PROVINCE           0
TP_PROVINCE                     295
REGION_NM                         0
REG_FACT_FL                       0
FACT_POST_FL                      0
REG_POST_FL                       0
REG_FACT_POST_FL                  0
REG_FACT_POST_TP_FL               0
FL_PRESENCE_FL              

До разбиения
- "1365-1368" - пенсионеры. Не импутировать. Сделать новые категории "Не указано"
- Первоначально удалить в ORG_TP_FCAPITAL значения "Без участия" для 'SOCSTATUS_PENS_FL = 1 (Пенсионер) и GEN_INDUSTRY = NaN. Опечатка
- Пропуски "1365-1368" заменить на "Не указано"
- Пропуски WORK_TIME для 'SOCSTATUS_PENS_FL = 1 (Пенсионер) заменить на "0"
- Пропуски TP_PROVINCE (область торговой точки, где клиент брал последний кредит) - заменить на значение области фактического пребывания FACT_ADDRESS_PROVINCE
- Пропуски PREVIOUS_CARD_NUM_UTILIZED (количество уже утилизированных карт (если пусто - 0))заменить на "0"  

После разбиения
- WORK_TIME. Обнулить всё, что больше 720, затем заменить средним после разбиения(см. выше)
- Из материалов "Модуль1". WORK_TIME > 720 заменить медианой после разбиения

- Пропуски WORK_TIME для 'SOCSTATUS_PENS_FL = 0 (Не пенсионер) заменить средним после разбиения
- Из материалов "Модуль1". Пропуски WORK_TIME для 'SOCSTATUS_PENS_FL = 0 (Не пенсионер) заменить медианой после разбиения

In [52]:
#Посмотреть, нет ли опечаток, странных значений в категориальных переменных
cat_columns = df.select_dtypes(include = ['object']).columns.tolist()
for col in cat_columns:
    print(col, df[col].unique())

EDUCATION ['Среднее специальное' 'Среднее' 'Неполное среднее' 'Высшее'
 'Неоконченное высшее' 'Два и более высших образования' 'Ученая степень']
MARITAL_STATUS ['Состою в браке' 'Гражданский брак' 'Разведен(а)' 'Не состоял в браке'
 'Вдовец/Вдова']
GEN_INDUSTRY ['Торговля' 'Информационные технологии' 'Образование'
 'Государственная служба' 'Другие сферы' 'Сельское хозяйство'
 'Здравоохранение' 'Металлургия/Промышленность/Машиностроение' nan
 'Коммунальное хоз-во/Дорожные службы' 'Строительство' 'Транспорт'
 'Банк/Финансы' 'Ресторанный бизнес/Общественное питание' 'Страхование'
 'Нефтегазовая промышленность' 'СМИ/Реклама/PR-агенства' 'Энергетика'
 'Салоны красоты и здоровья' 'ЧОП/Детективная д-ть'
 'Развлечения/Искусство' 'Наука' 'Химия/Парфюмерия/Фармацевтика'
 'Сборочные производства' 'Туризм'
 'Юридические услуги/нотариальные услуги' 'Маркетинг' 'Подбор персонала'
 'Информационные услуги' 'Недвижимость' 'Управляющая компания' 'Логистика']
GEN_TITLE ['Рабочий' 'Специалист' 'Руководите

In [53]:
#Удалить дубликаты
df.drop_duplicates(subset = None, keep = 'first', inplace=True)

In [54]:
len(df)

15223

In [55]:
#Дубликатов нет

In [56]:
# Сделать замены пропусков на "Не указано"

In [57]:
df['ORG_TP_FCAPITAL'] = np.where((df['ORG_TP_FCAPITAL'] == "Без участия")&(df['GEN_INDUSTRY'].isnull())&(df['SOCSTATUS_PENS_FL'] == 1), 
                                                                "Не указано", df['ORG_TP_FCAPITAL'])

In [58]:
column_list = ['GEN_INDUSTRY', 'GEN_TITLE', 'ORG_TP_STATE', 'ORG_TP_FCAPITAL', 'JOB_DIR']

In [59]:
for col in column_list:
    df[col] = np.where((df[col].isnull())&(df['SOCSTATUS_PENS_FL'] == 1), "Не указано",df[col])

In [60]:
for col in column_list:
    df[col] = np.where((df[col].isnull())&(df['SOCSTATUS_PENS_FL'] == 0), df[col].mode(),df[col])

In [61]:
# Поиск редких категорий
for col in cat_columns:
    print(df[col].value_counts(dropna = False))

Среднее специальное               6518
Среднее                           4679
Высшее                            3133
Неоконченное высшее                532
Неполное среднее                   340
Два и более высших образования      20
Ученая степень                       1
Name: EDUCATION, dtype: int64
Состою в браке        9416
Не состоял в браке    3622
Разведен(а)           1245
Вдовец/Вдова           588
Гражданский брак       352
Name: MARITAL_STATUS, dtype: int64
Торговля                                     2386
Другие сферы                                 1709
Не указано                                   1366
Металлургия/Промышленность/Машиностроение    1356
Государственная служба                       1286
Здравоохранение                              1177
Образование                                   998
Транспорт                                     787
Сельское хозяйство                            702
Строительство                                 574
Коммунальное хоз-во/Дорожны

In [62]:
#Замена редких категорий 
df.loc[df['EDUCATION'] == 'Ученая степень', 'EDUCATION'] = 'Два и более высших образования'
df.loc[df['GEN_INDUSTRY'] == 'Маркетинг', 'GEN_INDUSTRY'] = 'СМИ/Реклама/PR-агенства'
df.loc[df['JOB_DIR'] == 'Реклама и маркетинг', 'JOB_DIR'] = 'Участие в основ. деятельности'

In [63]:
#Пропуски WORK_TIME для 'SOCSTATUS_PENS_FL = 1 (Пенсионер) заменить на "0"
df['WORK_TIME'] = np.where((df['WORK_TIME'].isnull())&(df['SOCSTATUS_PENS_FL'] == 1), 0, df['WORK_TIME'])

In [64]:
#Пропуски TP_PROVINCE (область торговой точки, где клиент брал последний кредит) - заменить на 
#значение области фактического пребывания FACT_ADDRESS_PROVINCE
df['TP_PROVINCE'] = np.where(df['TP_PROVINCE'].isnull(), df['FACT_ADDRESS_PROVINCE'], df['TP_PROVINCE'])

In [65]:
#Пропуски PREVIOUS_CARD_NUM_UTILIZED (количество уже утилизированных карт (если пусто - 0))заменить на "0"
#Преобразовать в int
df['PREVIOUS_CARD_NUM_UTILIZED'] = np.where(df['PREVIOUS_CARD_NUM_UTILIZED'].isnull(), 0, df['PREVIOUS_CARD_NUM_UTILIZED'])
df['PREVIOUS_CARD_NUM_UTILIZED'] = df['PREVIOUS_CARD_NUM_UTILIZED'].astype('int64')

In [66]:
#Кто такой не пенсионер и без рабочего времени.  
df[(df['WORK_TIME'].isnull()) & (df['SOCSTATUS_PENS_FL'] == 0)]

,AGREEMENT_RK,TARGET,AGE,SOCSTATUS_WORK_FL,SOCSTATUS_PENS_FL,GENDER,CHILD_TOTAL,DEPENDANTS,EDUCATION,MARITAL_STATUS,GEN_INDUSTRY,GEN_TITLE,ORG_TP_STATE,ORG_TP_FCAPITAL,JOB_DIR,FAMILY_INCOME,PERSONAL_INCOME,REG_ADDRESS_PROVINCE,FACT_ADDRESS_PROVINCE,POSTAL_ADDRESS_PROVINCE,TP_PROVINCE,REGION_NM,REG_FACT_FL,FACT_POST_FL,REG_POST_FL,REG_FACT_POST_FL,REG_FACT_POST_TP_FL,FL_PRESENCE_FL,OWN_AUTO,AUTO_RUS_FL,HS_PRESENCE_FL,COT_PRESENCE_FL,GAR_PRESENCE_FL,LAND_PRESENCE_FL,CREDIT,TERM,FST_PAYMENT,DL_DOCUMENT_FL,GPF_DOCUMENT_FL,FACT_LIVING_TERM,WORK_TIME,FACT_PHONE_FL,REG_PHONE_FL,GEN_PHONE_FL,LOAN_NUM_TOTAL,LOAN_NUM_CLOSED,LOAN_NUM_PAYM,LOAN_DLQ_NUM,LOAN_MAX_DLQ,LOAN_AVG_DLQ_AMT,LOAN_MAX_DLQ_AMT,PREVIOUS_CARD_NUM_UTILIZED
7963,64469089,0,24,1,0,0,0,0,Среднее специальное,Не состоял в браке,Торговля,Специалист,Частная компания,Без участия,Участие в основ. деятельности,от 10000 до 20000 руб.,10000.0,Курская область,Курская область,Курская область,Курская область,ЦЕНТРАЛЬНЫЙ 1,1,1,1,1,1,0,0,0,0,0,0,0,5309.0,10,590.0,0,0,85,NaN,1,0,0,1,0,5,1,1,540.0,540.0,0


In [67]:
# Скорее всего, опечатка, т.к. указано "Частная компания". 
#Можно импутировать всего одно значение WORK_TIME и до разбиения, например, средним для 24-летних
df.loc[7963,'WORK_TIME'] = df[(df['AGE'] == 24)]['WORK_TIME'].mean()

In [68]:
df.isnull().sum()

In [69]:
##df.astype({'GEN_INDUSTRY':'str', 'GEN_TITLE':'str', 'ORG_TP_STATE':'str', 'ORG_TP_FCAPITAL':'str', 'JOB_DIR':'str'}).dtypes

In [70]:
##df.describe()

In [71]:
#на всякий случай сохраним перед удалением столбцов
df_copy = df.copy()

In [72]:
#dummi-кодирование (без которого никак при LogisticRegression) даст очень большое количество предиктов
#Можно предположить, что удалив пребывание, регистрацию, почту и место выдачи, не получим серьёзного искажения.
#Достаточно региона.
df_short = df.drop(columns = ['REG_ADDRESS_PROVINCE', 'FACT_ADDRESS_PROVINCE', 'POSTAL_ADDRESS_PROVINCE', 'TP_PROVINCE'])

In [73]:
train, test, y_train, y_test = train_test_split(df_short.drop('TARGET', axis=1), 
                                                df_short['TARGET'], 
                                                test_size=.3, 
                                                stratify=df_short['TARGET'], 
                                                random_state=155)

In [74]:
#WORK_TIME > 720 заменить медианой
train['WORK_TIME'] = np.where(train['WORK_TIME'] > 720, train['WORK_TIME'].median(), train['WORK_TIME'])
test['WORK_TIME'] = np.where(test['WORK_TIME'] > 720, test['WORK_TIME'].median(), test['WORK_TIME'])

In [75]:
train_onehot = pd.get_dummies(train)

In [76]:
test_onehot = pd.get_dummies(test)

In [77]:
#Сравним количество столбцов. Не получилось ли категорий, отсутствующих в test, но прсутствующих в train, 
#и наоборот
print(len (train_onehot.columns), len (test_onehot.columns))

128 128


In [78]:
# ЛучшеЭ наверное, глазами проверить 
train_onehot.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 10656 entries, 11143 to 15004
Data columns (total 128 columns):
 #   Column                                                  Non-Null Count  Dtype  
---  ------                                                  --------------  -----  
 0   AGREEMENT_RK                                            10656 non-null  int64  
 1   AGE                                                     10656 non-null  int64  
 2   SOCSTATUS_WORK_FL                                       10656 non-null  int64  
 3   SOCSTATUS_PENS_FL                                       10656 non-null  int64  
 4   GENDER                                                  10656 non-null  int64  
 5   CHILD_TOTAL                                             10656 non-null  int64  
 6   DEPENDANTS                                              10656 non-null  int64  
 7   PERSONAL_INCOME                                         10656 non-null  float64
 8   REG_FACT_FL                    

In [79]:
test_onehot.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 4567 entries, 13336 to 9107
Data columns (total 128 columns):
 #   Column                                                  Non-Null Count  Dtype  
---  ------                                                  --------------  -----  
 0   AGREEMENT_RK                                            4567 non-null   int64  
 1   AGE                                                     4567 non-null   int64  
 2   SOCSTATUS_WORK_FL                                       4567 non-null   int64  
 3   SOCSTATUS_PENS_FL                                       4567 non-null   int64  
 4   GENDER                                                  4567 non-null   int64  
 5   CHILD_TOTAL                                             4567 non-null   int64  
 6   DEPENDANTS                                              4567 non-null   int64  
 7   PERSONAL_INCOME                                         4567 non-null   float64
 8   REG_FACT_FL                      

In [80]:
standardscaler = StandardScaler()

In [81]:
standardscaler.fit(train_onehot)

C:\Users\Dell\Anaconda3\envs\py36\lib\site-packages\sklearn\preprocessing\data.py:625: DataConversionWarning: Data with input dtype uint8, int64, float64 were all converted to float64 by StandardScaler.
  return self.partial_fit(X, y)


StandardScaler(copy=True, with_mean=True, with_std=True)

In [82]:
train_standardscaled = standardscaler.transform(train_onehot)
test_standardscaled = standardscaler.transform(test_onehot)

C:\Users\Dell\Anaconda3\envs\py36\lib\site-packages\ipykernel_launcher.py:1: DataConversionWarning: Data with input dtype uint8, int64, float64 were all converted to float64 by StandardScaler.
  """Entry point for launching an IPython kernel.
C:\Users\Dell\Anaconda3\envs\py36\lib\site-packages\ipykernel_launcher.py:2: DataConversionWarning: Data with input dtype uint8, int64, float64 were all converted to float64 by StandardScaler.
  


In [83]:
logreg = LogisticRegression(solver = 'lbfgs', max_iter = 200)

In [84]:
logreg.fit(train_standardscaled, y_train)

LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=200, multi_class='warn',
          n_jobs=None, penalty='l2', random_state=None, solver='lbfgs',
          tol=0.0001, verbose=0, warm_start=False)

In [85]:
print(logreg.score(train_standardscaled, y_train))
print(logreg.score(test_standardscaled, y_test))

0.881006006006006
0.8780380994088023


In [86]:
logreg_predvalues = logreg.predict(test_standardscaled)
logreg_predvalues[:10]

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0], dtype=int64)

In [87]:
logreg_probabilities = logreg.predict_proba(test_standardscaled)
logreg_probabilities[:10]

array([[0.81323835, 0.18676165],
       [0.94345969, 0.05654031],
       [0.86521062, 0.13478938],
       [0.93745238, 0.06254762],
       [0.93163191, 0.06836809],
       [0.88700433, 0.11299567],
       [0.95373097, 0.04626903],
       [0.89509872, 0.10490128],
       [0.90363027, 0.09636973],
       [0.82652264, 0.17347736]])

In [88]:
accuracy_score(y_test, logreg_predvalues)

0.8780380994088023

In [89]:
logreg_probabilities_proba = logreg_probabilities[:,1]

In [90]:
roc_auc_score(y_test, logreg_probabilities_proba)

0.6986313074819785

In [91]:
#Попробовал бездумно GradientBoostingRegressor. Ерунда какая-то. Надо разбираться
Gradboost = GradientBoostingRegressor(max_depth = 6, random_state = 46)

In [92]:
Gradboost.fit(train_standardscaled, y_train)

GradientBoostingRegressor(alpha=0.9, criterion='friedman_mse', init=None,
             learning_rate=0.1, loss='ls', max_depth=6, max_features=None,
             max_leaf_nodes=None, min_impurity_decrease=0.0,
             min_impurity_split=None, min_samples_leaf=1,
             min_samples_split=2, min_weight_fraction_leaf=0.0,
             n_estimators=100, n_iter_no_change=None, presort='auto',
             random_state=46, subsample=1.0, tol=0.0001,
             validation_fraction=0.1, verbose=0, warm_start=False)

In [93]:
print(Gradboost.score(train_standardscaled, y_train))
print(Gradboost.score(test_standardscaled, y_test))

0.3979464131053607
0.022754307642173233


In [94]:
Gradboost_predvalues = Gradboost.predict(test_standardscaled)
Gradboost_predvalues.astype('int')
Gradboost_predvalues[:10]

array([0.21753029, 0.00362249, 0.09170282, 0.07372457, 0.01148759,
       0.07967145, 0.02227493, 0.21127808, 0.05509344, 0.20388918])